In [1]:
# permutations that require max number of dcj operations  - diameter of an instance 

# find the largest value of n which you can find the diameter 
# n --> diameter (randomly sample x number)
# increase n  

In [2]:
using Random

using NBInclude
@nbinclude("dcj_algo.ipynb")

SRC (.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.) --> TARGET (.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(g:t,f:h)(f:t,e:h)(e:t,.)

*************
DCJ Operation :: (d:h,e:t)(g:h,.) --> (d:h,g:h)(e:t,.)
number of operations = 1
--------------------------------------------------------------------------------------


1

In [3]:
# Function to get the first n letters of the alphabet
function first_n_letters(n::Int)
    return collect('a':'z')[1:n]
end

first_n_letters (generic function with 1 method)

In [24]:
# generates genomes/permutations/strings of length n 
# (up to 26 bc alphabet)
function generate_genome_str(n::Int)
    # generate first n letters
    letters = first_n_letters(n)

    # randomize capitalization (gene orientation)
    for i in length(letters) 
        if rand(Bool)
            letters[i] = uppercase(letters[i])
        end
    end 

    # shuffle letters to create permutation 
    letters = shuffle(letters)

    # randomly choose length of chromosomes and circular/linear 
    max_chrom_length = n 
    curr_letters_idx = 1

    genome_str = ""
    while max_chrom_length != 0
        # choose rand length up to max_chrom_length & decremenet max_chrom_length 
        chrom_length = rand(1:max_chrom_length)
        chrom = ""

        max_chrom_length -= chrom_length

        # choose linear or circ 
        linear = rand(Bool)

        # if linear, add telomeres 
        end_idx = curr_letters_idx + chrom_length - 1
        chrom = letters[curr_letters_idx:end_idx]
        if linear 
            chrom = "." * join(chrom) * "."
        end 
        
        chrom = join(chrom)
        # add comma after the last element if max_chrom_length != 0 
        if max_chrom_length != 0 
            genome_str = genome_str * chrom * ","
        else
            genome_str = genome_str * chrom 
            # insert commas to separate chromosomes 
        end 
        
        curr_letters_idx += chrom_length 
    end 

    return genome_str
end

generate_genome_str(5)

"a,.d.,.b.,cE"

In [ ]:
function sample_diameter(n::Int, num_src_permutations_to_generate::Int)
    x = 10  # hyperparam -- tune
    target = generate_genome_str(n) 
    max_dist = 0 
    src = generate_genome_str(n)
    max_dist_permutation = src

    for i in 1:num_src_permutations_to_generate

        dist = calculate_distance(src, target)
        if max_dist < dist 
            max_dist = dist
            max_dist_permutation = src 
        end 
        src = generate_permutation(n)
    end 

    return max_dist, max_dist_permutation
end 

function calculate_diameters(n::Int) 
    diameters = Vector{Tuple{Int, String}}()

    for i in 1:n
        # for x in 1:
        x = 1
        diam, max_dist_permutation = sample_diameter(i, x) 

        push!(diameters, (diam, max_dist_permutation))  # i == idx
    end 

    return diameters
end 

n = 1
calculate_diameters(n)

11

LoadError: MethodError: no method matching calculate_distance(::Nothing, ::Nothing)

In [ ]:
l = [1, 2, 3, 4]

l[2:3]

2-element Vector{Int64}:
 2
 3